# Import necessary packages

In [1]:
import pandas as pd
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../scripts')))
import  preprocessing as prep


# Load the data


In [2]:
fraud_data = pd.read_csv('../data/raw/Fraud_Data.csv')
ip_data = pd.read_csv('../data/raw/IpAddress_to_Country.csv')

# Handle missing values


In [3]:
fraud_data = prep.handle_missing_values(fraud_data)

# Data cleaning


In [4]:
fraud_data = prep.data_cleaning(fraud_data)

In [5]:
fraud_data.head()

,user_id,signup_time,purchase_time,purchase_value,device_id,source,browser,sex,age,ip_address,class
0,22058,2015-02-24 22:55:49,2015-04-18 02:47:11,34,QVPSPJUOCKZAR,SEO,Chrome,M,39,7.327584e+08,0
1,333320,2015-06-07 20:39:50,2015-06-08 01:38:54,16,EOGFQPIZPYXFZ,Ads,Chrome,F,53,3.503114e+08,0
2,1359,2015-01-01 18:52:44,2015-01-01 18:52:45,15,YSSKYOSJHPPLJ,SEO,Opera,M,53,2.621474e+09,1
3,150084,2015-04-28 21:13:25,2015-05-04 13:54:50,44,ATGTXKYKUDUQN,SEO,Safari,M,41,3.840542e+09,0
4,221365,2015-07-21 07:09:52,2015-09-09 18:40:53,39,NAUITBZFJKHWW,Ads,Safari,M,45,4.155831e+08,0


# Exploratory Data Analysis


In [6]:
prep.eda(fraud_data)

Univariate Analysis:
             user_id                    signup_time  \
count  151112.000000                         151112   
mean   200171.040970  2015-04-20 00:56:09.511329280   
min         2.000000            2015-01-01 00:00:42   
25%    100642.500000     2015-02-18 09:52:48.500000   
50%    199958.000000            2015-04-19 04:41:30   
75%    300054.000000  2015-06-18 14:47:22.750000128   
max    400000.000000            2015-08-18 04:40:29   
std    115369.285024                            NaN   

                       purchase_time  purchase_value            age  \
count                         151112   151112.000000  151112.000000   
mean   2015-06-16 02:56:38.759952896       36.935372      33.140704   
min              2015-01-01 00:00:44        9.000000      18.000000   
25%       2015-04-18 14:41:25.500000       22.000000      27.000000   
50%       2015-06-18 13:46:17.500000       35.000000      33.000000   
75%       2015-08-17 18:48:31.500000       49.000000     

# Merge datasets for geolocation analysis


In [7]:
merged_data = prep.merge_datasets(fraud_data, ip_data)

In [8]:
merged_data.head()

,user_id,signup_time,purchase_time,purchase_value,device_id,source,browser,sex,age,ip_address,class,ip_address_int,lower_bound_ip_address,upper_bound_ip_address,country,lower_bound_ip_address_int,upper_bound_ip_address_int
24727,99475,2015-01-15 22:52:56,2015-03-21 05:26:58,55,KKQVQFEISXTQI,Ads,IE,M,25,3.436292e+09,0,3436291840,3.436292e+09,3.436292e+09,Canada,3.436292e+09,3.436292e+09
32485,39268,2015-05-09 17:42:56,2015-06-04 05:13:18,36,LVTPXQBECONJX,SEO,IE,F,27,3.256946e+09,0,3256946176,3.256946e+09,3.256946e+09,Romania,3.256946e+09,3.256946e+09
35932,92325,2015-01-16 19:47:30,2015-05-03 17:19:26,27,IYIBXBGTIDAUY,Direct,Chrome,M,30,3.495475e+09,0,3495475200,3.495475e+09,3.495477e+09,Canada,3.495475e+09,3.495477e+09
37431,390559,2015-07-09 06:36:13,2015-09-22 04:07:00,51,SKNQFVNNGMSXY,Ads,Safari,M,24,1.613234e+09,0,1613234176,1.613234e+09,1.613365e+09,United States,1.613234e+09,1.613365e+09
52243,383733,2015-04-09 12:13:47,2015-07-22 23:48:13,42,PFHACSCWDLWUA,SEO,Chrome,F,45,3.469070e+09,0,3469070080,3.469070e+09,3.469070e+09,United States,3.469070e+09,3.469070e+09


# Feature engineering


In [9]:
engineered_data = prep.feature_engineering(merged_data)

# Normalization and Scaling


In [10]:
columns_to_scale = ['purchase_value', 'signup_to_purchase']
scaled_data = prep.normalize_and_scale(engineered_data, columns_to_scale)

# Encode categorical features


In [11]:
columns_to_encode = ['source', 'browser', 'sex']
encoded_data = prep.encode_categorical(scaled_data, columns_to_encode)

# View the processed data


In [12]:
encoded_data.head()

,user_id,signup_time,purchase_time,purchase_value,device_id,source,browser,sex,age,ip_address,class,ip_address_int,lower_bound_ip_address,upper_bound_ip_address,country,lower_bound_ip_address_int,upper_bound_ip_address_int,signup_to_purchase,hour_of_day,day_of_week
24727,99475,2015-01-15 22:52:56,2015-03-21 05:26:58,1.218471,KKQVQFEISXTQI,0,1,1,25,3.436292e+09,0,3436291840,3.436292e+09,3.436292e+09,Canada,3.436292e+09,3.436292e+09,-0.199243,5,5
32485,39268,2015-05-09 17:42:56,2015-06-04 05:13:18,-0.091960,LVTPXQBECONJX,2,1,0,27,3.256946e+09,0,3256946176,3.256946e+09,3.256946e+09,Romania,3.256946e+09,3.256946e+09,-1.527123,5,3
35932,92325,2015-01-16 19:47:30,2015-05-03 17:19:26,-0.712691,IYIBXBGTIDAUY,1,0,1,30,3.495475e+09,0,3495475200,3.495475e+09,3.495477e+09,Canada,3.495475e+09,3.495477e+09,1.259711,17,6
37431,390559,2015-07-09 06:36:13,2015-09-22 04:07:00,0.942591,SKNQFVNNGMSXY,0,2,1,24,1.613234e+09,0,1613234176,1.613234e+09,1.613365e+09,United States,1.613234e+09,1.613365e+09,0.164361,4,1
52243,383733,2015-04-09 12:13:47,2015-07-22 23:48:13,0.321860,PFHACSCWDLWUA,2,0,0,45,3.469070e+09,0,3469070080,3.469070e+09,3.469070e+09,United States,3.469070e+09,3.469070e+09,1.177051,23,2
